In [1]:
# change notes 11/22/24:

# moved data.yaml file to dataset folder per YOLO documentation
# dataset needs to be in same folder as notebook
# updated yaml.data to 10 classes, due to dataset error for only subset to 7 or 8
# added label dir splitting process
# added markdowns
# prevented raw_images and raw_labels from being cleared
# trained model from labels

WIP: 
<br> fix yaml location pull (yaml needs to be in same folder as images)
<br>any further fine tuning on hyperparams
<br>add validation plots,
<br>initial and predicted image subplots *from custom dataset only*
<br>live video feed,
<br>dashboard

# Import Libraries

In [5]:
# import modules
import os # file 
import shutil
import cv2 # opencv for images
import random
import numpy as np
import pandas as pd
import torch
from ultralytics import YOLO #for obeject detection
import albumentations as A # for image augmentation
from albumentations.pytorch import ToTensorV2 # for image formating
# from tqdm import tqdm  # to show processing progress
# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")

# Clear the "train", "test", "valid" folders from base dir

In [6]:
# create direcories to organize images and cleanup for a new to avoid duplicate images 
def reset_directories(root_dir, directories):
    """
    Check if the specified directories exist. If they do, delete them and recreate them.
    Ensures the directories are clean before use.

    Parameters:
        directories (list): List of directories to reset.
    """
    for path in directories:
        dir_path = root_dir+path
        if os.path.exists(dir_path):
            # delete the directory and all its contents
            try:
                shutil.rmtree(dir_path)
                print(f"Deleted existing directory: {dir_path}")
            except Exception as e:
                print(f"Failed to delete {dir_path}. Reason: {e}")
        
        # Recreate the directory
        try:
            os.makedirs(dir_path, exist_ok=True)
            print(f"Recreated directory: {dir_path}")
        except Exception as e:
            print(f"Failed to create directory {dir_path}. Reason: {e}")
# base dir
dataset_base_dir = "./datasets"

# Define directories to reset
directories_to_reset = [
    "/images/train",
    "/images/valid",
    "/images/test",
    "/labels/train",
    "/labels/valid",
    "/labels/test"
]

# Reset directories
reset_directories(dataset_base_dir, directories_to_reset)

print("Directories reset and ready for use.")

Deleted existing directory: ./datasets/images/train
Recreated directory: ./datasets/images/train
Deleted existing directory: ./datasets/images/valid
Recreated directory: ./datasets/images/valid
Deleted existing directory: ./datasets/images/test
Recreated directory: ./datasets/images/test
Deleted existing directory: ./datasets/labels/train
Recreated directory: ./datasets/labels/train
Deleted existing directory: ./datasets/labels/valid
Recreated directory: ./datasets/labels/valid
Deleted existing directory: ./datasets/labels/test
Recreated directory: ./datasets/labels/test
Directories reset and ready for use.


# Split files from "raw_image" and "raw_label" folders
#### (Note: "./datasets/raw_images" and "./datasets/raw_labels" folders must already contain all images and all labels)

In [7]:
# Set random seed for reproducibility
random.seed(42)

# source directory containing all raw images and labels
source_image_dir = "./datasets/raw_images"
source_label_dir = "./datasets/raw_labels"

# base directory for the organized dataset
base_dir = dataset_base_dir

image_dirs = {
    "train": os.path.join(base_dir, "images/train"),
    "valid": os.path.join(base_dir, "images/valid"),
    "test": os.path.join(base_dir, "images/test")
}
label_dirs = {
    "train": os.path.join(base_dir, "labels/train"),
    "valid": os.path.join(base_dir, "labels/valid"),
    "test": os.path.join(base_dir, "labels/test")
}

# Ensure directories exist
for dir_path in list(image_dirs.values()) + list(label_dirs.values()):
    os.makedirs(dir_path, exist_ok=True)

# Extract the image files
image_files = [f for f in os.listdir(source_image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(image_files)

# Create DataFrame with file paths and dataset split assignments
df = pd.DataFrame({"filename": image_files})

# Dynamically split dataset into train, valid, and test
train_size = int(0.7 * len(df))  # 70% for training
valid_size = int(0.2 * len(df))  # 20% for validation
test_size = len(df) - train_size - valid_size  # remaining 10% for testing

train_labels = ["train"] * train_size
valid_labels = ["valid"] * valid_size
test_labels = ["test"] * test_size

# Combine and assign to DataFrame
df["split"] = train_labels + valid_labels + test_labels

# Copy files to respective folders
for _, row in df.iterrows():
    img_file = row["filename"]
    split = row["split"]
    
    # Source paths
    img_src = os.path.join(source_image_dir, img_file)
    label_src = os.path.join(source_label_dir, os.path.splitext(img_file)[0] + ".txt")
    
    # Destination paths
    img_dest = os.path.join(image_dirs[split], img_file)
    label_dest = os.path.join(label_dirs[split], os.path.splitext(img_file)[0] + ".txt")
    
    # Copy image file
    if os.path.exists(img_src):
        shutil.copy(img_src, img_dest)
    
    # Copy corresponding label file
    if os.path.exists(label_src):
        shutil.copy(label_src, label_dest)

# Training

In [8]:
model = YOLO('yolov8n.pt')
model.train(data='data.yaml', epochs = 10, val = True)

Ultralytics 8.3.32  Python-3.12.4 torch-2.5.1+cpu CPU (Intel Core(TM) i9-14900KF)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train56, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

train: Scanning C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\labels\test.cache... 287 im

train: WARNING  C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\images\test\002654_jpg.rf.ff1deb021121bfef83e8a9152093596e.jpg: ignoring corrupt image/label: Label class 9 exceeds dataset class count 8. Possible class labels are 0-7
train: WARNING  C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\images\test\003184_jpg.rf.ddcabf097616516ad576f90948a0ac0f.jpg: ignoring corrupt image/label: Label class 9 exceeds dataset class count 8. Possible class labels are 0-7
train: WARNING  C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\images\test\005577_jpg.rf.28a70bd7ad7c4a004b19903de1eafc9e.jpg: ignoring corrupt image/label: Label class 9 exceeds dataset class count 8. Possible class labels are 0-7
train: WARNING  C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\images\test\1288788-une-employee-aide-des-voyageurs-en-provenance-de-chine-le-26-janvier-2020-a-l-aeroport-de-roissy_jpg.rf.9


val: Scanning C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\labels\valid.cache... 571 ima

val: WARNING  C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\images\valid\-2252-_png_jpg.rf.a71a293e8fb2e65c9b851ee278fbc4c0.jpg: ignoring corrupt image/label: Label class 9 exceeds dataset class count 8. Possible class labels are 0-7
val: WARNING  C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\images\valid\-3394-_png_jpg.rf.a0f81aeb67bec81f0a77c518490205bc.jpg: ignoring corrupt image/label: Label class 9 exceeds dataset class count 8. Possible class labels are 0-7
val: WARNING  C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\images\valid\005100_jpg.rf.2994d16a99c03162e7b756fa13fdd3c5.jpg: ignoring corrupt image/label: Label class 9 exceeds dataset class count 8. Possible class labels are 0-7
val: WARNING  C:\Users\DARiN\Documents\Python-JL\Github\aai-501-final-project_dv\datasets\images\valid\005139_jpg.rf.b51d392a15c3ca4943cfe750427ceb97.jpg: ignoring corrupt image/label: Label class 9 exceeds datas

...

                   all        405       4979       0.47      0.365      0.371      0.208



10 epochs completed in 0.111 hours.
Optimizer stripped from runs\detect\train56\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train56\weights\best.pt, 6.2MB

Validating runs\detect\train56\weights\best.pt...
Ultralytics 8.3.32  Python-3.12.4 torch-2.5.1+cpu CPU (Intel Core(TM) i9-14900KF)
Model summary (fused): 168 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:16


                   all        405       4979      0.471      0.365      0.371      0.208
               Hardhat        181        493       0.74       0.49      0.537       0.29
                  Mask        177        244       0.69      0.463      0.521      0.289
            NO-Hardhat        216        355       0.35      0.268       0.23      0.107
               NO-Mask        223        484      0.247      0.107     0.0817     0.0264
        NO-Safety Vest        292        612      0.443      0.227      0.259      0.135
                Person        385       1456      0.665      0.594      0.641       0.42
           Safety Cone         44        116      0.188      0.147     0.0966     0.0304
           Safety Vest        183        445      0.386      0.324      0.313      0.146
             Machinery        311        774      0.527      0.668      0.657      0.426
Speed: 0.9ms preprocess, 26.1ms inference, 0.0ms loss, 9.3ms postprocess per image
Results saved to runs\detec

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000211AA029040>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.

In [ ]:
# load YOLOv8 pre-trained model
model = YOLO("../models/yolov8n.pt")

# define augmentation pipeline
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0, 0, 0), std=(1, 1, 1)),  # No change to pixel values
    ToTensorV2()  # convert to PyTorch tensor 
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

# define function to annotate and process images
def annotate_images(df, split, image_dirs, label_dirs, confidence_threshold=0.5):
    data = []
    split_df = df[df["split"] == split]
    for _, row in split_df.iterrows():
        image_file = row["filename"]
        # load the image
        image_path = os.path.join(source_image_dir, image_file)
        image = cv2.imread(image_path)

        # perform inference using YOLOv8
        results = model(image_path)

        # extract bounding boxes and labels
        bboxes = []
        labels = []
        for result in results[0].boxes:
            box = result.xywhn[0].cpu().numpy()  # Normalized x_center, y_center, width, height
            class_id = int(result.cls[0].cpu().numpy())
            confidence = float(result.conf[0].cpu().numpy())

            # filter by confidence threshold
            if confidence >= confidence_threshold:
                bboxes.append(box.tolist())
                labels.append(class_id)

                # add annotation details to the df list
                data.append({
                    "filename": image_file,
                    "split": split,
                    "class_id": class_id,
                    "confidence": confidence,
                    "x_center": box[0],
                    "y_center": box[1],
                    "width": box[2],
                    "height": box[3],
                })

        # apply augmentation
        if bboxes:  # Only augment if there are bounding boxes
            augmented = augmentation(image=image, bboxes=bboxes, labels=labels)
            image = augmented["image"]
            bboxes = augmented["bboxes"]
            labels = augmented["labels"]

        # convert to numpy format for saving 
        if isinstance(image, torch.Tensor):  # If tensor, convert to numpy for opencv
            image = image.permute(1, 2, 0).cpu().numpy() # changes the order of the tensor dimensions from (C, H, W) (Channel-Height-Width, common in PyTorch) to (H, W, C) (Height-Width-Channel, required by OpenCV and most image libraries).
            image = (image * 255).astype(np.uint8)  # Convert to uint8 for OpenCV

        # save the image to the appropriate directory
        output_image_path = os.path.join(image_dirs[split], image_file)
        cv2.imwrite(output_image_path, image)

        # set YOLO format labels
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(label_dirs[split], label_file)

        # create YOLO format label for the corresponding image
        with open(label_path, "w") as f:
            for bbox, class_id in zip(bboxes, labels):
                # Write each valid detection to the file in YOLO format
                f.write(f"{class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

    return data
# annotate and process images for all splits
annotation_data = []
for split in ["train", "valid", "test"]:
    # append each list to annotation data
    annotation_data.extend(annotate_images(df, split, image_dirs, label_dirs, confidence_threshold=0.5))


print("Annotation completed and dataset organized into train, validation, and test directories.")

In [ ]:
annotation_df = pd.DataFrame(annotation_data)
annotation_df.head()